### VERGE: Vector-Mode Regional Geospatial Encoding
# Initial Embeddings

Back in the "02" folder, we trained a Masked Geospatial Model.
That model can compute a set of embeddings for any tile,
for which the inputs are a set of geospatial entities.
These embeddings are permutation-equivariant ("perm-e")
with respect to the input features,
which is not what we want for a regional embedding.
In this folder we are building
a fully permutation-invariant ("perm-i") aggregation of the perm-e
outputs.

In this notebook, we compute those perm-e embeddings for all instances
in our training and validation datasets.
That is, we run the model that was trained bac inthe 02 folder.


## Processing Setup

In [1]:
# Google colab setup
import os
from google.colab import drive
drive.mount('/content/drive')
project_home = '/content/drive/MyDrive/Projects/verge'
os.chdir(project_home)
!pip install geo_encodings

Mounted at /content/drive


In [2]:
# Local processing setup
# project_home = '..'

## Notebook Setup

In [3]:
import pandas as pd
import numpy as np
import glob
import pickle
import os
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader
import copy
import json
from geo_encodings import MPPEncoder

import sys
sys.path.append(project_home)
from utils.geo_transformer_mem import VergeDataset, verge_collate_fn, GeospatialTransformer


## Parameters

In [4]:
# The name of the ROI to use.
roi_name = 'newengland'

# The name of the general-purpose data directory.
data_home = '%s/data' % (project_home)

# The name of the ROI-specific data directory.
roi_home = '%s/data/%s' % (project_home, roi_name)

# The unique identifier of the model to be used.
run_id = '201b'

# Identifier of the splits file to use.
splits_id = '201'

# What type of device to train on.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device', device)


using device cuda


## Preliminaries

In [5]:
# Read the ROI definition.
fname = '%s/roi.json' % roi_home
with open(fname) as source:
    roi = json.load(source)

tile_size = roi['tile_size']
encoding_resolution = roi['encoding_resolution']

# We need the dimension of the encoding.
encoder = MPPEncoder(
    region=[0, 0, tile_size, tile_size],
    resolution=encoding_resolution,
    center=True
)
geo_encoding_dim = len(encoder)
print('%d elements in encodings' % geo_encoding_dim)


400 elements in encodings


In [6]:
# Read the list of labels.
fname = '%s/labels.csv' % data_home
labels = pd.read_csv(fname)
n_classes = len(labels)
print('%d labels in this dataset' % n_classes)

label_id_lookup = {
    z['label']: z['id']
    for z in labels.to_dict('records')
}

label_name_lookup = {
    z['id']: z['label']
    for z in labels.to_dict('records')
}

22 labels in this dataset


In [7]:
# Read the file that gives class probabilities.
fname = '%s/class_info.csv' % roi_home
class_info = pd.read_csv(fname)
print('%d class info records' % len(class_info))

22 class info records


## Load data
We determine which files to read by loading the associated "split" file.

In [8]:
# Get the list of AOI tags. They can be found in the splits file.
fname = '%s/models/splits-%s.csv' % (roi_home, splits_id)
splits = pd.read_csv(fname)
aoi_tags = np.unique(splits['aoi_tag'])

## Prep model and data

In [9]:
# The dataset constructor requires a lookup table for class probabilities.
class_prob_lookup = {
    z['label']: z['prob']
    for z in class_info.to_dict('records')
}

In [10]:
# Load the model.
model_fname = '%s/models/model-%s' % (roi_home, run_id)
model = torch.load(model_fname, weights_only=False)
print('loaded %s' % model_fname)

model.to(device)
model.eval()

loaded /content/drive/MyDrive/Projects/verge/data/newengland/models/model-201b


GeospatialTransformer(
  (input_proj): Linear(in_features=422, out_features=128, bias=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (output_head): Linear(in_features=128, out_features=22, bias=True)
)

In [11]:
# Loop over encoded input files. For each one, define a dataset,
# run it through the model, and generate an initial set of embeddings.
for aoi_tag in aoi_tags:

    encoding_fname = '%s/encodings/%s.pkl' % (roi_home, aoi_tag)
    print('\n', encoding_fname)

    # Define a dataset and datloader for this input file.
    # Note that we set the batch size to 1. This effectively removes all
    # padding, as the dataloader pads to the largest object
    # in the batch.
    dataset = VergeDataset([encoding_fname], n_classes, mask_fraction=0.0, class_prob=class_prob_lookup)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=True,
        collate_fn=verge_collate_fn,
        drop_last=False
    )

    # Get embeddings for every tile in this AOI.
    embeddings_for_this_aoi = []
    for features, labels, attention_mask, idents in dataloader:
        features = features.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        embeddings = model.embed(features, attention_mask)
        embeddings_for_this_aoi.append({
            'aoi_tag': idents[0].split(':')[0],
            'tile_tag': idents[0].split(':')[1],
            'embedding': embeddings
        })

    # Save those.
    ofname = '%s/initials/%s.pkl' % (roi_home, aoi_tag)
    os.makedirs(os.path.dirname(ofname), exist_ok=True)
    with open(ofname, 'wb') as dest:
        pickle.dump(embeddings_for_this_aoi, dest)
    print('wrote %s' % ofname)



 /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0687w-449n.pkl
loaded 37 instances from /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0687w-449n.pkl
wrote /content/drive/MyDrive/Projects/verge/data/newengland/initials/0687w-449n.pkl

 /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0688w-447n.pkl
loaded 29 instances from /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0688w-447n.pkl
wrote /content/drive/MyDrive/Projects/verge/data/newengland/initials/0688w-447n.pkl

 /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0688w-448n.pkl
loaded 37 instances from /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0688w-448n.pkl
wrote /content/drive/MyDrive/Projects/verge/data/newengland/initials/0688w-448n.pkl

 /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0689w-447n.pkl
loaded 36 instances from /content/drive/MyDrive/Projects/verge/data/newengland/encodings/0689w-447n.pkl
wrote /c

## QA